# Embedding Model with Instructions

## IMDB Dataset

The IMDB dataset is a large dataset of movie reviews used for sentiment analysis. It contains 50,000 reviews labeled as positive or negative, which can be used for binary sentiment classification tasks. 

- **Train**: Contains 25,000 labeled reviews for training
- **Test**: Contains 25,000 labeled reviews for evaluating

### Save Data to disk
- Retrieve Dataset
- Save (sampled) dataset to disk

In [1]:
from datasets import load_dataset

def save_dataset(random_sample_size=5000):  # Reduce sample size for testing
    # Load the dataset
    dataset = load_dataset('stanfordnlp/imdb')
    
    # Access the train, test splits
    train_dataset = dataset['train']
    test_dataset = dataset['test']

    # Random sample the dataset, only use random_sample_size
    train_dataset = train_dataset.shuffle(seed=42).select(range(random_sample_size))
    test_dataset = test_dataset.shuffle(seed=42).select(range(random_sample_size))

    train_dataset.save_to_disk('sampled_train_dataset')
    test_dataset.save_to_disk('sampled_test_dataset')
save_dataset()

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

### Load Local Dataset
- Load dataset from disk

In [2]:
from datasets import load_from_disk
try:
    train_dataset = load_from_disk('sampled_train_dataset')
    test_dataset = load_from_disk('sampled_test_dataset')
    
    print(">>Train Dataset loaded<<")
    # Print a sample from the loaded datasets to verify
    print("Text:", train_dataset[0]["text"], "\nLabel:", train_dataset[0]["label"])
    
    print("\n>>Test Dataset loaded<<")
    # Print a sample from the loaded test datasets to verify
    print("Text:", test_dataset[0]["text"], "\nLabel:", test_dataset[0]["label"])
except Exception as e:
    print(f"An error occurred while loading the datasets: {e}")

>>Train Dataset loaded<<
Text: There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all... 
Label: 1

>>Test Dataset loaded<<
Text: <br /><br />When I unsuspectedly rented A Thousand Acres, I thought I was in for an entertaining King Lear story and of course Michelle Pfeiffer was in it, so what could

## Text Embedding Models

- **BERT** (Bidirectional Encoder Representations from Transformers)
    - Retrieved from [Hugging Face](https://huggingface.co/google-bert/bert-base-uncased)

- **Sentence Transformer** (all-MiniLM-L12-v2)
    - Retrieved from [Hugging Face](https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2)

- **T5** (Text-To-Text Transfer Transformer)
    - Retrieved from [Hugging Face](https://huggingface.co/google-t5/t5-base)

- **Instructor**
    - Retrieved from [Hugging Face](https://huggingface.co/hkunlp/instructor-large)

In [3]:
import torch
from transformers import pipeline
device = 0 if torch.cuda.is_available() else -1

model_BERT = pipeline("feature-extraction", model="google-bert/bert-base-uncased", device=device)
#model_ST = pipeline("feature-extraction", model="sentence-transformers/all-MiniLM-L12-v2", device=device)
#model_T5 = pipeline("feature-extraction", model="google-t5/t5-base", device=device)
#model_INS = pipeline("feature-extraction", model="hkunlp/instructor-large", device=device)

- Helper function using the provided embedding model to encode text
- returns Array of Tensors and labels

In [4]:
from tqdm.auto import tqdm
from transformers.pipelines.pt_utils import KeyDataset
import numpy as np

def process_dataset(model, dataset, key, truncation=False, padding=False, max_length=0, use_mean_pooling=False):
    data = KeyDataset(dataset, key)
    pipe = model(data, return_tensors=True, truncation=truncation, padding=padding, max_length=max_length)
    embeddings=[]
    for tensor in tqdm(pipe, desc="Encoding text"): 
        # Use mean pooling to 1 dim for classification
        if(use_mean_pooling):
            tensor = tensor.mean(dim=1)
            tensor = tensor.flatten()
        embeddings.append(tensor)
    return np.array(embeddings), np.array(dataset["label"])

In [5]:
import textwrap

def modify_dataset(example):
    example['text'] = "Prefix: " + example['text'] + " :Suffix"
    return example

# Apply the function to the dataset
train_dataset_aug = train_dataset.map(modify_dataset)

# Check the first review
example = train_dataset_aug['text'][0]
print(textwrap.fill(example, width=60))

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Prefix: There is no relation at all between Fortier and
Profiler but the fact that both are police series about
violent crimes. Profiler looks crispy, Fortier looks
classic. Profiler plots are quite simple. Fortier's plot are
far more complicated... Fortier looks more like Prime
Suspect, if we have to spot similarities... The main
character is weak and weirdo, but have "clairvoyance".
People like to compare, to judge, to evaluate. How about
just enjoying? Funny thing too, people writing Fortier looks
American but, on the other hand, arguing they prefer
American series (!!!). Maybe it's the language, or the
spirit, but I think this series is more English than
American. By the way, the actors are really good and funny.
The acting is not superficial at all... :Suffix


Embed `train_dataset` and `test_dataset` then save them to disk
- CPU 10 it/s
- GPU 100 it/s

Augment dataset with instructions

In [6]:
train_embeddings, train_labels = process_dataset(dataset=train_dataset, model=model_BERT, key="text", truncation=True, padding=True, max_length=512,use_mean_pooling=True)

Encoding text:   0%|          | 0/5000 [00:00<?, ?it/s]

c:\Users\Voice\anaconda3\envs\embed\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


KeyboardInterrupt: 

This is not required to run if loading from disk

In [24]:
from datetime import datetime
import os, json
def save_embeddings(embeddings, model_name, save_path="data"):
    timestamp = datetime.now().strftime("%m-%d_%H:%M")
    # Calculate the average shape of tensors
    tensor_shapes = [tensor.shape for tensor in embeddings]
    avg_shape = np.mean(tensor_shapes, axis=0).tolist()
    
    embedding_info = {
        'model_name': model_name,
        'num_embeddings': len(embeddings),
        'avg_embedding_shape': avg_shape,
        'created_at': timestamp
    }
    
    os.makedirs(save_path, exist_ok=True)
    embedding_file = os.path.join(save_path, f"{model_name}_embeddings.npy")
    metadata_file = os.path.join(save_path, f"{model_name}_metadata.json")
    
    np.save(embedding_file, embeddings)

    with open(metadata_file, 'w') as f:
        json.dump(embedding_info, f)
    
    print(f"Embeddings and metadata saved for {model_name} at {timestamp}")

In [28]:
def load_embeddings(embedding_file, metadata_file):
    embeddings = np.load(embedding_file)
    with open(metadata_file, 'r') as f:
        metadata = json.load(f)
    
    print(f"Loaded embeddings from model: {metadata['model_name']}")
    print(f"Number of embeddings: {metadata['num_embeddings']}")
    print(f"Average embedding shape: {metadata['avg_embedding_shape']}")
    print(f"Created at: {metadata['created_at']}")
    
    return embeddings

In [25]:
save_path="data"
model_name="BERT"
save_embeddings(train_embeddings, model_name, save_path)

Embeddings and metadata saved for BERT at 05-23_11:51


Load embedding from disk

In [29]:
save_path="data"
model_name="BERT"

embedding_file = os.path.join(save_path, f"{model_name}_embeddings.npy")
metadata_file = os.path.join(save_path, f"{model_name}_metadata.json")
load_embeddings(embedding_file,metadata_file)

Loaded embeddings from model: BERT
Number of embeddings: 5000
Average embedding shape: [768.0]
Created at: 05-23_11:51


array([[-0.11507314, -0.16738787,  0.28467953, ..., -0.12811592,
         0.5055427 ,  0.09243022],
       [-0.21617004, -0.00583607,  0.44736055, ..., -0.11052745,
         0.32436627, -0.13193743],
       [-0.12583718,  0.09763683,  0.0976319 , ...,  0.13283734,
         0.26076582, -0.00548352],
       ...,
       [-0.10729862, -0.1170555 ,  0.07840945, ..., -0.02886777,
         0.23762697,  0.08733881],
       [-0.16576715,  0.03112957,  0.17408013, ...,  0.02879209,
         0.20242864,  0.10291475],
       [ 0.08241627,  0.24093148,  0.37436956, ...,  0.00803341,
         0.26234606,  0.04334791]], dtype=float32)

### Train classifiers to evaluate Embedding performance
- Linear
    - SVM
- Non-Lienar
    - MLP

Train SVM model using `train_embeddings` and `train_labels`

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Train an SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(train_embeddings, train_labels)

Evaluate SVM results using `test_embeddings` and `test_labels`

In [12]:
# Predict and evaluate
predicted_labels = svm_model.predict(test_embeddings)
print(classification_report(y_true=test_labels, y_pred=predicted_labels))

              precision    recall  f1-score   support

           0       0.90      0.91      0.90     12500
           1       0.91      0.90      0.90     12500

    accuracy                           0.90     25000
   macro avg       0.90      0.90      0.90     25000
weighted avg       0.90      0.90      0.90     25000



MLP

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

# Define the MLP model
mlp_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, alpha=1e-4,
                          solver='sgd', verbose=1, random_state=1,
                          learning_rate_init=.1)

# Train the MLP model
mlp_model.fit(train_embeddings, train_labels)

# Predict and evaluate
predicted_labels_mlp = mlp_model.predict(test_embeddings)
print(classification_report(y_true=test_labels, y_pred=predicted_labels_mlp))


Iteration 1, loss = 0.45508312
Iteration 2, loss = 0.32360067
Iteration 3, loss = 0.30201054
Iteration 4, loss = 0.29376630
Iteration 5, loss = 0.29093661
Iteration 6, loss = 0.28485199
Iteration 7, loss = 0.28306261
Iteration 8, loss = 0.27774261
Iteration 9, loss = 0.28060523
Iteration 10, loss = 0.27239479
Iteration 11, loss = 0.26746099
Iteration 12, loss = 0.26639452
Iteration 13, loss = 0.26759380
Iteration 14, loss = 0.27147793
Iteration 15, loss = 0.26649923
Iteration 16, loss = 0.26131166
Iteration 17, loss = 0.26058729
Iteration 18, loss = 0.25630766
Iteration 19, loss = 0.25358914
Iteration 20, loss = 0.25397461
Iteration 21, loss = 0.25147743
Iteration 22, loss = 0.24943971
Iteration 23, loss = 0.25084300
Iteration 24, loss = 0.24741396
Iteration 25, loss = 0.24528136
Iteration 26, loss = 0.24756351
Iteration 27, loss = 0.24111311
Iteration 28, loss = 0.24670886
Iteration 29, loss = 0.23864409
Iteration 30, loss = 0.23626259
Iteration 31, loss = 0.23756441
Iteration 32, los

## The Text Embedding Pipeline

In [1]:
from TextEmbeddingPipeline import EmbedFlow

flow = EmbedFlow(model_name="BERT", dataset_name="IMDB", prefix="Prefix: ", suffix=" :Suffix")

flow.start_flow(sample_size=1000, use_mean_pooling=True, method='svm')


Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Data loaded and sampled.
Local data loaded.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Data augmented with prefixes and suffixes.


Embedding text:   0%|          | 0/1000 [00:00<?, ?it/s]

c:\Users\Voice\anaconda3\envs\embed\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Embeddings and labels saved at 05-27_09:10.
Data loaded for model: BERT
SVM Evaluation Report:
              precision    recall  f1-score   support

           0       0.88      0.89      0.89       104
           1       0.88      0.86      0.87        96

    accuracy                           0.88       200
   macro avg       0.88      0.88      0.88       200
weighted avg       0.88      0.88      0.88       200

